# Growth Coupling Suite - Example use case
Find strain designs that couple the succinate exchange reaction to growth in a reduced E. coli model

In [1]:
from growth_coupling_suite.gcOpt_algorithm import gcOpt
from growth_coupling_suite.strain_analysis.strain_design_analysis import StrainDesignAnalyzer
import cobra

# Load and check the config file

In [2]:
import gcOpt_config_file as config

## Load and prepare the model

In [3]:
# load "mid-scale" metabolic model of E. coli (https://doi.org/10.1038/srep39647)
model_name = "ECC2.json"
model = cobra.io.load_json_model(model_name)

# prepare model
# set glucose uptake rate
model.exchanges.EX_glc__D_e.lower_bound = -10
# set oxygen uptake rate
model.exchanges.EX_o2_e.lower_bound = -20
# disable co2 uptake
model.exchanges.EX_co2_e.lower_bound = 0

Academic license - for non-commercial use only - expires 2022-08-07
Using license file C:\Users\Tobi\gurobi.lic


## Define the target reaction

In [4]:
#  define target reaction -> succinate exchange
target_reaction = "EX_succ_e"

# preapre gcOpt config
config.exchanges_not_to_add.append(target_reaction)  

# Run a single gcOpt instance
The solver can still use multiple threads

In [5]:
# load gcOpt class
if __name__ == "__main__":
    GCS = gcOpt.GCOpt(model, target_reaction, config=config, build_gcopt_problem=True)
    # solve gcOpt MILP problem
    GCS.optimize(init_DesignAnalyzer=False)

Read LP format model from file C:\Users\Tobi\AppData\Local\Temp\tmplxsxn0fd.lp
Reading time = 0.01 seconds
: 510 rows, 1002 columns, 4546 nonzeros
Load existing heterologous reaction database model...
	ECC2_hr_database
Read LP format model from file C:\Users\Tobi\AppData\Local\Temp\tmpq_6pax71.lp
Reading time = 0.03 seconds
: 3225 rows, 7442 columns, 35388 nonzeros
Update heterologous reaction database...
	Reactions of iAF692 already in heterologous reaction database
	Reactions of iAF987 already in heterologous reaction database
	Reactions of iAM_Pb448 already in heterologous reaction database
	Reactions of iCHOv1 already in heterologous reaction database
	Reactions of iCN718 already in heterologous reaction database
	Reactions of iCN900 already in heterologous reaction database
	Reactions of iEK1008 already in heterologous reaction database
	Reactions of iHN637 already in heterologous reaction database
	Reactions of iIS312 already in heterologous reaction database
	Reactions of iIT341

# Run multiple gcOpt instances in parallel/sequentially
- Can be useful since the Gurobi solver doesn't make use of all available threads anymore after running the MILP problem for some time.
- It is recommended to start parallel computations from the shell, Anaconda prompt, etc.
- NOTE: Due to the use of parallel worker no output may be displayed!

In [7]:
# set up config parameters
# Any config file attribute can be added to the parameter dictionaries 
parameters_parallel = [
    {"num_total_interventions": 3, "num_deletions": 2, "num_addins":2, "output_file_id": "i3_d2_a2_cf1_cs1"},
    {"num_total_interventions": 6, "num_deletions": 5, "num_addins":5, "output_file_id": "i6_d5_a5_cf1_cs1"},
    {"num_total_interventions": 9, "num_deletions": 8, "num_addins":8, "output_file_id": "i9_d8_a8_cf1_cs1"},
]
# "output_suffix" is added to the "output_file_id"
parameters_sequential = [
    {"growth_rate_fix": 0.1, "output_suffix": "gr_01"},
    {"growth_rate_fix": 0.3, "output_suffix": "gr_03"},
    {"growth_rate_fix": 0.6, "output_suffix": "gr_06"},
]


# initialize gcOpt class
# run gcOpt in parallel
if __name__ == "__main__":
    # load gcOpt class
    GCS = gcOpt.GCOpt(model, target_reaction, config=config, build_gcopt_problem=False)
    # run gcOpt
    GCS.optimize_series(
        parameters_parallel=parameters_parallel,
        parameters_sequential=parameters_sequential,
        max_workers = 3, # number of parallel workers, compare with the number of processes allocated in the config file to avoid overloading your machine
        init_DesignAnalyzer=False
    )

Read LP format model from file C:\Users\Tobi\AppData\Local\Temp\tmp2jdt5q6r.lp
Reading time = 0.01 seconds
: 1726 rows, 3089 columns, 13961 nonzeros


# Analyze, save, and plot all results

In [9]:
if __name__ == "__main__": # not necessary if "GCS.optimize_series()" is not called before    
    sda = StrainDesignAnalyzer()
    
    sda.load_strain_design_files_from_dir(
        config.output_dir,
        eval_gpr=True
        )
    
    sda.growth_coupling_summary(
        results_filename="gcOpt_summary_result",
        results_dir=config.output_dir,
        determine_significant_designs=True, # duplicate solutions are disregarded,
                                            # design objects (deletions, add-ins, etc.) that do not contribute to the coupling are stripped from solution
        save_results=True,
        save_flux_space_plots=True,
        eval_gpr=True
    )

Read LP format model from file C:\Users\Tobi\AppData\Local\Temp\tmp114y6qoq.lp
Reading time = 0.02 seconds
: 1509 rows, 3814 columns, 17778 nonzeros
Read LP format model from file C:\Users\Tobi\AppData\Local\Temp\tmp_gmu76nl.lp
Reading time = 0.01 seconds
: 510 rows, 1002 columns, 4546 nonzeros
Load file: C:\Users\Tobi\Github\Strainware-m-FORK\growth_coupling_suite\example\results\gcOpt_solution_dict_EX_succ_e_i5_k4_a4_cs1_cf1.pickle
Read LP format model from file C:\Users\Tobi\AppData\Local\Temp\tmpvlauoax1.lp
Reading time = 0.01 seconds
: 510 rows, 1002 columns, 4546 nonzeros
Read LP format model from file C:\Users\Tobi\AppData\Local\Temp\tmpdf_w8g0d.lp
Reading time = 0.01 seconds
: 510 rows, 1002 columns, 4546 nonzeros
Reverse previous strain design...
Reverse previous strain design...
Apply parsed strain design (design_1)...
	Set medium composition
	Set design interventions
0.3128427715599902


C:\ProgramData\Anaconda3\envs\env_gcs\lib\site-packages\pandas\core\internals\blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


Reverse previous strain design...
Apply parsed strain design (design_2)...
	Set medium composition
	Set design interventions
0.4702937230200093
Reverse previous strain design...
Apply parsed strain design (design_3)...
	Set medium composition
	Set design interventions
2.912359218123278
Reverse previous strain design...
Apply parsed strain design (design_4)...
	Set medium composition
	Set design interventions
3.031297853369921
Read LP format model from file C:\Users\Tobi\AppData\Local\Temp\tmp3_s2avy6.lp
Reading time = 0.01 seconds
: 512 rows, 1004 columns, 4548 nonzeros
Read LP format model from file C:\Users\Tobi\AppData\Local\Temp\tmpg0f5fgyh.lp
Reading time = 0.01 seconds
: 510 rows, 1002 columns, 4546 nonzeros
Read LP format model from file C:\Users\Tobi\AppData\Local\Temp\tmpik37exxw.lp
Reading time = 0.02 seconds
: 1509 rows, 3814 columns, 17778 nonzeros
Reverse previous strain design...
Apply parsed strain design (design_1)...
	Set medium composition
	Set design interventions
Ta